In [ ]:
import json
import textwrap
from collections import defaultdict
from datasets import load_dataset
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

In [3]:
dataset = load_dataset("inparallel/saudinewsnet", trust_remote_code=True)
print(dataset)
sample = dataset['train'][0]
print(sample)

DatasetDict({
    train: Dataset({
        features: ['source', 'url', 'date_extracted', 'title', 'author', 'content'],
        num_rows: 31030
    })
})
{'source': 'aawsat', 'url': 'http://aawsat.com/home/article/410826/بريطانيا-أربعة-محاور-لاستراتيجية-جديدة-تتصدى-للتطرف-على-مدى-خمس-سنوات', 'date_extracted': '2015-07-21 02:51:32', 'title': 'بريطانيا: أربعة محاور لاستراتيجية جديدة تتصدى للتطرف على مدى خمس سنوات', 'author': 'لندن: رنيم حنوش', 'content': 'حدد رئيس الوزراء البريطاني ديفيد كاميرون، اليوم (الاثنين)، ملامح استراتيجية للتصدي للتطرف داخل بريطانيا؛ وهي مسألة اعتبرها كاميرون "صراع جيلنا"، متعهدا خلال خطابه في مدينة بيرمنغهام بالتصدي لهؤلاء الذين ينشرون التطرف بين الشبان المسلمين البريطانيين.\n\n ورسم كاميرون الاطار العام لاستراتيجية مكافحة التطرف التي المقرر ان تنشر كاملة في وقت لاحق هذا العام، والتي تسعى للتصدي لانتشار الأفكار المتطرفة التي يروج لها متشددو تنظيم "داعش".\n\n وحسبما تناقلت وسائل الإعلام البريطانية، فإن خطة رئيس الوزراء ستكون على مدى خمسة أعوام للقضاء على التطرف ا

In [4]:
MODEL_NAME = "CAMeL-Lab/bert-base-arabic-camelbert-mix-ner"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)

ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"  
)

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [5]:
def chunk_text(text, max_length=450):
    return textwrap.wrap(text, width=max_length)

def extract_named_entities(text):
    chunks = chunk_text(text)  
    all_entities = []

    for chunk in chunks:
        entities = ner_pipeline(chunk)  
        all_entities.extend(entities)  

    structured_entities = defaultdict(list)
    
    for entity in all_entities:
        entity_type = entity.get("entity_group", entity.get("entity", "UNKNOWN"))  
        entity_text = entity["word"].replace("##", "")  

        if entity_text not in structured_entities[entity_type]: 
            structured_entities[entity_type].append(entity_text)

    return dict(structured_entities)

ner_results = []
for i in range(100):  
    article = dataset["train"][i]  
    if isinstance(article, dict) and "content" in article:
        entities = extract_named_entities(article["content"])
        ner_results.append({
            "title": article.get("title", "No Title"),
            "content": article.get("content", "No Content"),
            "source": article.get("source", "Unknown Source"),
            "date_extracted": article.get("date_extracted", "Unknown Date"),
            "entities": entities
        })

    else:
        print(f"Skipping invalid article: {article}")  

with open("arabic.json", "w", encoding="utf-8") as f:
    json.dump(ner_results, f, ensure_ascii=False, indent=4)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
